In [6]:
# import numpy as np
# def symmetric(mat):
#     size = 32
#     mat1, mat2, mat3, mat4, mat5, mat6, mat7 = np.zeros((size, size, size)),np.zeros((size, size, size)),np.zeros((size, size, size)),np.zeros((size, size, size)),np.zeros((size, size, size)),np.zeros((size, size, size)),np.zeros((size, size, size))
#     for i in range(size):
#         for j in range(size):
#             for k in range(size):
#                 mat1[i][j][k] = mat[size-1-i][j][k]
#                 mat2[i][j][k] = mat[size-1-i][size-1-j][k]
#                 mat3[i][j][k] = mat[size-1-i][size-1-j][size-1-k]
#                 mat4[i][j][k] = mat[size-1-i][j][size-1-k]
#                 mat5[i][j][k] = mat[i][size-1-j][size-1-k]
#                 mat6[i][j][k] = mat[i][size-1-j][k]
#                 mat7[i][j][k] = mat[i][j][size-1-k]
#     X.extend([mat, mat1, mat2, mat3, mat4, mat5, mat6, mat7])
    
# def dataEnhance(mat):
#     size = 32
#     mat1 = np.zeros((size, size, size))
#     mat2 = np.zeros((size, size, size))
#     mat3 = np.zeros((size, size, size))
#     mat4 = np.zeros((size, size, size))
#     mat5 = np.zeros((size, size, size))
#     for i in range(size):
#         for j in range(size):
#             for k in range(size):
#                 mat1[i][j][k] = mat[i][k][j]
#                 mat2[i][j][k] = mat[k][i][j]
#                 mat3[i][j][k] = mat[k][j][i]
#                 mat4[i][j][k] = mat[j][i][k]
#                 mat5[i][j][k] = mat[j][k][i]
#     symmetric(mat)
#     symmetric(mat1)
#     symmetric(mat2)
#     symmetric(mat3)
#     symmetric(mat4)
#     symmetric(mat5)

# 不用运行，jupyter在后面模块中识别不了这段代码

# import numpy as np
# def symmetric(mat):
#     size = 32
#     mat1, mat2, mat3, mat4, mat5, mat6, mat7 = np.zeros((size, size, size)),np.zeros((size, size, size)),np.zeros((size, size, size)),np.zeros((size, size, size)),np.zeros((size, size, size)),np.zeros((size, size, size)),np.zeros((size, size, size))
#     for i in range(size):
#         for j in range(size):
#             for k in range(size):
#                 mat1[i][j][k] = mat[size-1-i][j][k]
#                 mat2[i][j][k] = mat[size-1-i][size-1-j][k]
#                 mat3[i][j][k] = mat[size-1-i][size-1-j][size-1-k]
#                 mat4[i][j][k] = mat[size-1-i][j][size-1-k]
#                 mat5[i][j][k] = mat[i][size-1-j][size-1-k]
#                 mat6[i][j][k] = mat[i][size-1-j][k]
#                 mat7[i][j][k] = mat[i][j][size-1-k]
#     X.append(mat)
#     y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
#     X.append(mat1)
#     y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
#     X.append(mat2)
#     y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
#     X.append(mat3)
#     y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
#     X.append(mat4)
#     y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
#     X.append(mat5)
#     y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
#     X.append(mat6)
#     y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
#     X.append(mat7)
#     y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
# def dataEnhance(mat):
#     size = 32
#     symmetric(mat)
#     mat = mat.swapaxes(1, 2)
#     symmetric(mat)
#     mat = mat.swapaxes(0, 1)
#     symmetric(mat)
#     mat = mat.swapaxes(1, 2)
#     symmetric(mat)
#     mat = mat.swapaxes(0, 1)
#     symmetric(mat)
#     mat = mat.swapaxes(1, 2)
#     symmetric(mat)


### 以上是数据增强的测试代码，待检验

##### for some cosmologies data are missing...!!!??? we have to skip them

In [2]:
#import os 
#os.environ['CUDA_VISIBLE_DEVICES']='1'
%pylab inline
import numpy as np
import keras, os, pynbody, struct

lsstr = "ls /media/cosmo/Seagate/cola_multiverse/om_As/"

def cosmostr(om, As):
    return 'om%.3f' % om + '_As%.3f' % As

def snpfiles(cosmology, snpstr='c'):  #
    return os.popen(lsstr + cosmology + "*snap*" + snpstr + ".*").read().split()

def gridfiles(cosmology, snpstr='c'):
    return os.popen(lsstr + cosmology + "*grid*" + snpstr + ".*").read().split()

def mocklist():
    files = os.popen(lsstr + "om*.lua").read().split('\n')
    # *代替多个字母,即列出所有符合条件的.lua文件:om...
    cosmologies = []  # 宇宙学参数
    mocks = {}  # 模拟测试
    ifile = 0  # 有效文件
    for nowfile in files:
        # str[a:b]不存在时,返回'',不存在则忽略
        nowstr = nowfile[-39:-10]
        if nowstr == '':
            continue
        cosmologies.append(nowstr[0:15])
        ifile += 1
        try:
            mocks[nowstr[0:15]] = {'om': float(nowstr[2:7]), 'As': float(nowstr[10:15]),
                                   'sigma8': float(nowstr[23:29])}
            # 添加随机数种子
            ranseed = float(open(nowfile, 'r').readline().split()[2])  # 默认以所有空字符为分隔符,包括空格,\n,\t
            mocks[nowstr[0:15]]['ranseed'] = int(ranseed)
            # print(ranseed)
        except:
            pass
    return cosmologies, files, mocks

gridfile_dict = {}

cosmologies, filenames, infos = mocklist()
print('In total ', len(cosmologies), 'cosmologies')

print('Build up gridfile_dict... (for speed-up of load_grid()) ')
for cosmology in cosmologies:
    rlt = gridfiles(cosmology)
    if rlt == []:
        print ('\tmissing cosmology!', cosmology)
    else:
        gridfile_dict[cosmology] = rlt[0]

Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


In total  465 cosmologies
Build up gridfile_dict... (for speed-up of load_grid()) 
	missing cosmology! om0.180_As2.040
	missing cosmology! om0.180_As2.100
	missing cosmology! om0.180_As2.120
	missing cosmology! om0.180_As2.140
	missing cosmology! om0.180_As2.200
	missing cosmology! om0.180_As2.220
	missing cosmology! om0.180_As2.260


In [3]:
def load_grid(gridfile, snpstr='c', printinfo=False):  # 网格加载
    #gridfile = os.popen(lsstr + cosmology+"_sigma8_*grid*" + snpstr + ".*").read().split()[0]
    #print('load in gridfile : ', gridfile, '...')
    nowf = open(gridfile, 'rb')  # 以二进制形式读取文件
    # struct:对python基本类型值与用python字符串格式表示的C struct类型间转化
    size = struct.unpack('f' * 1, nowf.read(4 * 1))[0]
    grid_nc = struct.unpack('i' * 1, nowf.read(4 * 1))[0]
    data = struct.unpack('f' * grid_nc ** 3, nowf.read(4 * grid_nc ** 3))
    if printinfo:
        print('read in box size     \n\t', size)
        print('read in num_grid      \n\t', grid_nc)
        print('read in coarse grid \n\tsize    : ', len(data), '\n\texpect  : ', grid_nc ** 3)
    nowf.close()
    return np.array(data).reshape((grid_nc, grid_nc, grid_nc))

def subcubes(A):  # the X data
    rlt = []
    for row1 in [0, 32, 64]:
        for row2 in [0, 32, 64]:
            for row3 in [0, 32, 64]:
                rlt.append(A[row1:row1+32,row2:row2+32,row3:row3+32])
    return rlt

def data_augument(A):
    rlt = []


In [15]:
# lsstr = "ls /media/minstrel/Seagate/cola_multiverse/om_As/"
# 初始化数据集,因函数定义中参数不可转为全局变量
# 此过程受train_test_split的test_size参数的影响,则今后更改或维护程序应注意
test_size = 0.3
batch_size = 4  # size : 15 <= size < 31
# 20:53-54s, 11:49s
num_subcube = 64
num_data_augument = 48

x_test = np.zeros((int(test_size*batch_size)+1,32,32,32, 1))
y_test = np.zeros((int(test_size*batch_size)+1, 2))
# x_test = np.zeros((int(test_size*batch_size)+1,32,32,32, 1))
# y_test = np.zeros((int(test_size*batch_size)+1, 2))


def train_generator():  # 必须无限循环yield数据,全部数据遍历后再重新遍历数据,为下一个epoch yield 数据
    def symmetric(mat):
        size = 32
        mat1, mat2, mat3, mat4, mat5, mat6, mat7 = np.zeros((size, size, size)),np.zeros((size, size, size)),np.zeros((size, size, size)),np.zeros((size, size, size)),np.zeros((size, size, size)),np.zeros((size, size, size)),np.zeros((size, size, size))
        for i in range(size):
            for j in range(size):
                for k in range(size):
                    mat1[i][j][k] = mat[size-1-i][j][k]
                    mat2[i][j][k] = mat[size-1-i][size-1-j][k]
                    mat3[i][j][k] = mat[size-1-i][size-1-j][size-1-k]
                    mat4[i][j][k] = mat[size-1-i][j][size-1-k]
                    mat5[i][j][k] = mat[i][size-1-j][size-1-k]
                    mat6[i][j][k] = mat[i][size-1-j][k]
                    mat7[i][j][k] = mat[i][j][size-1-k]
        X.append(mat)
        y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
        X.append(mat1)
        y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
        X.append(mat2)
        y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
        X.append(mat3)
        y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
        X.append(mat4)
        y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
        X.append(mat5)
        y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
        X.append(mat6)
        y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
        X.append(mat7)
        y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
    def dataEnhance(mat):
        size = 32
        symmetric(mat)
        mat = mat.swapaxes(1, 2)
        symmetric(mat)
        mat = mat.swapaxes(0, 1)
        symmetric(mat)
        mat = mat.swapaxes(1, 2)
        symmetric(mat)
        mat = mat.swapaxes(0, 1)
        symmetric(mat)
        mat = mat.swapaxes(1, 2)
        symmetric(mat)
    i = 0
    while 1:
        X = []
        y = []
        global x_test, y_test, test_size, batch_size
        for cosmology in cosmologies[batch_size * i: batch_size * (i + 1)]:
            try:
                gridfile = gridfile_dict[cosmology]
                griddata = load_grid(gridfile, 'c')
                for subcube in subcubes(griddata):
                    dataEnhance(subcube)   # X添加48个数据
#                     X.append(subcube)
#                     y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
                    
            except:
                pass
        X = np.array(X)
        y = np.array(y)
        x_train, x_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=test_size)
        x_train = x_train.reshape(-1, 32, 32, 32, 1)
        x_test = x_test.reshape(-1, 32, 32, 32, 1)
        i += 1
        yield x_train, y_train
        # 15个批次后重新遍历数据,此循环即死循环
        if i == 465//batch_size:
            i = 0


In [16]:
from keras import Sequential, layers
from sklearn import model_selection, metrics

model1 = keras.Sequential([
    layers.BatchNormalization( input_shape=(32, 32, 32, 1)),
    layers.Conv3D(32, (3, 3, 3), activation='relu'),
    layers.AveragePooling3D(pool_size=(2, 2, 2)),
    layers.BatchNormalization(),
    layers.Conv3D(64, (3, 3, 3), activation='relu'),
    layers.AveragePooling3D(pool_size=(2, 2, 2)),
    layers.BatchNormalization(),
    layers.Conv3D(128, (3, 3, 3), activation='relu'),
    layers.AveragePooling3D(pool_size=(2, 2, 2)),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(2, ),
])
model1.compile(optimizer=keras.optimizers.Adadelta(), loss='mean_squared_error',
              metrics=['mean_squared_error'])



# *************************************************************************************************
model1.fit_generator(train_generator(),
                    steps_per_epoch=465//batch_size,
                    epochs=50,
                    verbose=1,
                    validation_data=(x_test, y_test))


Epoch 1/50


ResourceExhaustedError: OOM when allocating tensor with shape[3628,30,30,30,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node conv3d_16/convolution}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss_5/mul}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
import time
from keras import Sequential, layers
from sklearn import model_selection, metrics

start = time.clock()
test_size = 0.3
batch_size = 15  # size : 15 <= size < 31
num_subcube = 64
num_data_augument = 48

x_test = np.zeros((int(test_size*batch_size)+1,32,32,32, 1))
# print("first dimension", int(test_size*batch_size*48*27)+1)
y_test = np.zeros((int(test_size*batch_size)+1, 2))
X = []
y = []
for cosmology in cosmologies[1:2]:
    try:
        gridfile = gridfile_dict[cosmology]
        griddata = load_grid(gridfile, 'c')
        for subcube in subcubes(griddata):
#             X.append(subcube)
#             y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
            dataEnhance(subcube)
            for j in range(48):
                        y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
            # print(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]).shape)
            
    except:
        pass

X = np.array(X)
y = np.array(y)
print(len(X))
print(len(y))
x_train, x_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=test_size)
x_train = x_train.reshape(-1, 32, 32, 32, 1)
x_test = x_test.reshape(-1, 32, 32, 32, 1)
print("yTest:",y_test.shape)
print("xTest:",x_test.shape)
print("xTrain:",x_train.shape)
print("yTrain:",y_train.shape)
print(int(test_size*batch_size)+1)
elapsed = (time.clock() - start)
print("Time used:",elapsed)

## Test section

In [ ]:
def create_validate_sample(nsample, use_random=True, startid=None ):
    cosmologies = list(gridfile_dict); ncosmo = len(cosmologies)
    if use_random:
        rows = [np.random.randint(0,ncosmo) for row in range(nsample)];
        rows = list(set(rows))
        while len(rows) < nsample:
            rows = rows + [np.random.randint(0,ncosmo) for row in range(nsample - len(rows))];
            rows = list(set(rows))
    else:
        rows = range(startid, startid+nsample)
    x, y =[], []
    for row in rows:
        cosmology = cosmologies[row]
        gridfile = gridfile_dict[cosmology]
        griddata = load_grid(gridfile)
        for subcube in subcubes(griddata):
            x.append(subcube)
            y.append(np.array([infos[cosmology]['om'], infos[cosmology]['sigma8']]))
    x = np.array(x); x = x.reshape(-1, 32, 32, 32, 1); y = np.array(y)
    return x, y

def plot_test(model, x, y, plot_avg_predict = True, fig=None, ax = None, plot_subpoints=False):
    y_predict = model.predict(x); 
    
    if fig == None or ax ==None:
        fig, ax = subplots(figsize=(14,6))
    cs = range(len(y)); cs = cs / mean(cs)
    ax.scatter(y[:,0], y[:,1], c='b', marker='*', label='input', s=200)
    if plot_subpoints:
        ax.scatter(y_predict[:,0], y_predict[:,1], c='g',  marker='p', s=50, label='outputs')
    
        
    om_test, w_test = y[:,0], y[:,1]
    om_predict, w_predict = y_predict[:,0], y_predict[:,1]
    
    if plot_avg_predict:
        ax.scatter(mean(y_predict[:,0]), mean(y_predict[:,1]), marker='*', c='r', label='output_avg', s=200)
        ax.plot([om_test[0], mean(y_predict[:,0])], [w_test[0], mean(y_predict[:,1])], lw=2, c='k', ls='--' )

    
    #for row in range(len(om_predict)):
    #    ax.plot( [om_predict[row], om_test[row]], [w_predict[row], w_test[row]], lw=0.5, c='gray' )
    ax.set_xlabel(r'$\Omega_m$',fontsize=16); ax.set_ylabel(r'$w$',fontsize=16)
    ax.legend()   
    return fig, ax

##### model1, 10 epochs: Don't run it! (参数已改不可恢复）

In [ ]:
fig, ax = None, None
for row in range(10):
    x_test, y_test = create_validate_sample(1, use_random=True, startid=row)
    fig, ax = plot_test(model1, x_test, y_test, fig=fig, ax=ax)
ax.grid(); plt.show()

##### 50 epochs`5

In [ ]:
fig, ax = None, None
for row in range(10):
    x_test, y_test = create_validate_sample(1, use_random=True, startid=row)
    fig, ax = plot_test(model1, x_test, y_test, fig=fig, ax=ax)
ax.grid(); plt.show()

##### model2, 10 epochs: Don't run it! (参数已改不可恢复）

In [ ]:
fig, ax = None, None
for row in range(10):
    x_test, y_test = create_validate_sample(1, use_random=True, startid=row)
    fig, ax = plot_test(model2, x_test, y_test, fig=fig, ax=ax)
ax.grid(); plt.show()

##### 50 epochs

In [ ]:
fig, ax = None, None
for row in range(10):
    x_test, y_test = create_validate_sample(1, use_random=True, startid=row)
    fig, ax = plot_test(model2, x_test, y_test, fig=fig, ax=ax)
ax.grid(); plt.show()